# Ensemble methods 

## Motivacija


Pretpostavimo da ste filmski redatelj i napravili ste kratki film o vrlo važnoj i zanimljivoj temi. Prije objavljivanja filma želite dobiti povratne informacije (ocjene). Koji su mogući načini na koje to možete učiniti?


**1.** *Možete zamoliti nekog od vaših prijatelja da ocijeni vaš film.*  
Sasvim je moguće da vas osoba koju ste odabrali jako voli i ne želi vam slomiti srce davajući lošu ocjenu.


**2.** *Drugi način može biti ako zamolite 5 vaših kolega da ocijene film.*  
Ovaj način bi trebao pružiti bolju ideju o filmu odnosno može dati poštene ocjene za film, no problem i dalje postoji. Ovih 5 osoba možda neće biti “stručnjaci za temu” vašeg filma. Naravno, mogli bi razumjeti kinematografiju, snimke ili zvuk, ali u isto vrijeme možda i nisu najbolji suci tamnog humora.


**3.** *Kako bi bilo da zamolite 50 ljudi da ocijene film?*  
Neki od njih mogu biti vaši prijatelji, neki od njih mogu biti vaši kolege, a neki čak i potpuni stranci.

Odgovori bi, u ovom slučaju, bili općenitiji i raznolikiji jer sada imate ljude s različitim skupinama vještina. I kako se ispostavilo - ovo je bolji pristup za dobivanje poštenih ocjena od prethodnih slučajeva koje smo vidjeli.

Pomoću ovih primjera možete zaključiti da će raznolika skupina ljudi vjerojatno donijeti bolje odluke u odnosu na pojedince. Slično vrijedi i za različite modele u usporedbi s pojedinim modelima. Ova raznolikost u strojnom učenju postiže se tehnikom nazvanom Ensemble Learning.


---




## Prije početka potrebno je ponoviti nekoliko osnovnih pojmova:

---
### Varijanca (engl. *variance*)
Očekivano kvadratno odstupanje slučajne varijable od njezine srednje vrijednosti. Dana je formulom: $$\sigma^2 = \frac{\sum(x - \mu)^2}{N}$$ gdje je: 
 - $\sigma^2$ varijanca
 - $x$ vrijednost slučajne varijable
 - $\mu$ aritmetička sredina podataka
 - $N$ broj opservacija
 
 
 
### Sistemska pogreška procjenitelja (engl. *bias of an estimator*)
Razlika između očekivane vrijednosti procjenitelja i stvarne vrijednosti parametra koji se procjenjuje.



### Bootstraping 
Odvija se na sljedeći način: Neka imamo uzorak $X$ veličine $N$. Možemo napraviti novi uzorak iz izvornog uzorka izvlačenjem $N$ elemenata iz posljednjeg uzorka nasumično i jednoliko, uz zamjenu. Drugim riječima, odabiremo slučajni element iz izvornog uzorka veličine $N$ i to $N$ puta. Jednako je vjerojatno da će svi elementi biti odabrani, tako da je svaki element odabran s jednakom vjerojatnošću $\frac{1}{N}$. 
 
Recimo da izvlačimo kuglice iz vrećice jednu po jednu. U svakom koraku se odabrana kuglica se vraća natrag u vrećicu tako da je sljedeći odabir izveden sa jednakom vjerojatnošću, tj. iz istog broja kuglica $N$. Potrebno je imati na umu da, budući da se kuglice vraćaju, se mogu pojaviti duplikati u novom uzorku. Nazovimo ovaj novi uzorak $X_1$.

Ponavljajući ovu proceduru $M$ puta, kreiramo $M$ *bootstrap* uzoraka $X1,…, XM$. Na kraju, imamo dovoljan broj uzoraka i možemo izračunati različite statistike izvorne distribucije.

<div style="width:70%;margin:0 auto;">![Boostraping.png](img/boostraping.png)</div>
 
---
 

# Općenito


Cilj orkestriranih metoda (engl. *Ensemble methods*) je kombinirati predviđanja nekoliko osnovnih procjenitelja (engl. *estimators*) izgrađenih s danim algoritmom učenja kako bi se poboljšala generalizabilnost / robustnost u usporedbi sa jednim procjeniteljem.

<div style="width:90%;margin:0 auto;">![ensemble_machine_learning.png](img/ensemble_machine_learning.png)</div>

---
Obično se razlikuju dvije obitelji *ensemble* metoda:

- **Averaging methods**: glavni princip je izgradnja nekoliko neovisnih procjenitelja iz kojih se kod predviđanja izračunava srednja vrijednost. Kombinirani procjenitelji su u prosjeku bolji nego bilo koji pojedinačni procjenitelj jer je njegova varijanca smanjena.


- **Boosting methods**: bazni procjenitelji se slijedno izgrađuju pokušavajući smanjiti sistemsku pogrešku (*bias*) kombiniranog procjenitelja. Motivacija je kombinirati nekoliko slabih modela kako bi se izgradio snažan procjenitelj.


---

### Tri najpopularnije metode za kombiniranje predviđanja različitih modela su:

- **Voting methods**: glavni princip je izgradnja nekoliko neovisnih procjenitelja, tipično različitih tipova, koji predviđanjem vrijednosti daju *glas* toj vrijednosti.


- **Bagging methods**: Izgradnja višestrukih modela (obično istog tipa) iz različitih poduzoraka niza podataka za trening. 


- **Boostring methods**: Izgradnja više modela (obično istog tipa) od kojih svaki uči popraviti pogreške predviđanja prethodnog modela u lancu.

---

## Dataset koji će se koristiti kod primjera

---

Dataset se može preuzeti i [ovdje](https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii/) (potrebna registracija)


<pre>
<span style="border-bottom: 1px solid #d8d8d8;display: inline-block;width: 501px;"><b>Variable</b>           Description</span>
<b>Loan_ID</b>            Unique Loan ID
<b>Gender</b>             Male/ Female
<b>Married</b>            Applicant married (Y/N)
<b>Dependents</b>         Number of dependents
<b>Education</b>          Applicant Education (Graduate/ Under Graduate)
<b>Self_Employed</b>      Self employed (Y/N)
<b>ApplicantIncome</b>    Applicant income
<b>CoapplicantIncome</b>  Coapplicant income
<b>LoanAmount</b>         Loan amount in thousands
<b>Loan_Amount_Term</b>   Term of loan in months
<b>Credit_History</b>     credit history meets guidelines
<b>Property_Area</b>      Urban/ Semi Urban/ Rural
<b>Loan_Status</b>        Loan approved (Y/N)
</pre>

In [46]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Učitaj dataset
dataset = pd.read_csv('train.csv')

# Loan_ID je jedinstven pa samim time ne pridonosi modelu
dataset = dataset.drop(labels="Loan_ID", axis=1, inplace=False)

# Popunjavanje nedostajućih vrijednosti
dataset['Gender'] = dataset['Gender'].fillna('Male')
dataset['Married'] = dataset['Married'].fillna('Yes')
dataset['Dependents'] = dataset['Dependents'].fillna('0')
dataset['Self_Employed'] = dataset['Self_Employed'].fillna('No')
dataset['LoanAmount'] = dataset['LoanAmount'].fillna(dataset['LoanAmount'].mean())
dataset['Loan_Amount_Term'] = dataset['Loan_Amount_Term'].fillna(360.0)
dataset['Credit_History'] = dataset['Credit_History'].fillna(1.0)

train, test = train_test_split(dataset, test_size=0.2, random_state=0)

x_train = train.drop('Loan_Status', axis=1)
y_train = train['Loan_Status']

x_test = test.drop('Loan_Status', axis=1)
y_test = test['Loan_Status']

# one-hot encoding
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)


x_train_r = train.drop('LoanAmount', axis=1)
y_train_r = train['LoanAmount']

x_test_r = test.drop('LoanAmount', axis=1)
y_test_r = test['LoanAmount']

# one-hot encoding
x_train_r = pd.get_dummies(x_train_r)
x_test_r = pd.get_dummies(x_test_r)


# 1. Voting Classifier

Ideja iza *VotingClassifier*-a je kombinirati konceptualno različite klasifikatore i koristiti većinski glas ili srednju predviđenu vjerojatnost (*soft vote*) da bi se predvidjele klase. Takav klasifikator može biti koristan za skup klasifikatora sa sličnim performansama kako bi se izjednačile slabosti pojedinačnih klasifikatora



## 1.1 Major Class Labels (Major/Hard Voting)


U većinskom glasanju, predviđena klasa za pojedini uzorak je klasa koja predstavlja većinu (**mod**) predviđenih klasa od svakog pojedinog klasifikatora



In [47]:
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(random_state=1)
model2 = tree.DecisionTreeClassifier(random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

model.fit(x_train, y_train)
model.score(x_test, y_test)


0.66666666666666663

## 1.2 Weighted Average Probabilities (Soft Voting)

Za razliku od većinskog glasanja (*hard voting*), *soft voting* vraća klasu koja proizlazi kao maksimum od sume predviđenih vjerojatnosti. Specifične težine se mogu dodijeliti svakom klasifikatoru kroz `weights` parametar. Kada su težine dodijeljene, prikupljaju se vjerojatnosti klase za svaki klasifikator, množe se sa njegovom težinom i zatim izračunata prosječna vrijednost. Konačna klasa je izvedena iz klase sa najvećom prosječnom vjerojatnosti.

In [48]:
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(random_state=1)
model2 = tree.DecisionTreeClassifier(random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='soft', weights=[2,1])

model.fit(x_train,y_train)
model.score(x_test,y_test)

0.7967479674796748

# 2.Bagging methods

Pretpostavimo da imamo skup $X$ za treniranje. Koristeći *bootstrapping* generiramo uzorke $X1,…, XM$. Sada, za svaki taj uzorak, treniramo vlastiti klasifikator $a_i(x)$. Konačni klasifikator će izračunati prosjek izlaza iz svih tih pojedinačnih klasifikatora. U slučaju klasifikacije, ova tehnika koristi glasovanje dano formulom: $a(x) = \frac{1}{M} \sum a_i(x)$


<div style="width:70%;margin:0 auto;">![bagging.png](img/bagging.png)</div>

---


## 2.1 Bagging meta-estimator


Bagging meta-estimator je *ensembling* algoritam koji se može koristiti i za klasifikacijske (BaggingClassifier) i regresijske (BaggingRegressor) probleme. Slijedi tipične *bagging* tehnike kako bi se napravila predviđanja.

U nastavku slijede koraci za *bagging meta-estimator* algoritam:

1. Stvaraju se slučajni podskupovi iz izvornog skupa podataka (bootstrapping).
2. Podskup skupa podataka uključuje sve značajke.
3. Na svakom od ovih manjih skupova izgrađen je, korisnički određen, bazni procjenitelj.
4. Predviđanja iz svakog modela se kombiniraju kako bi se dobio konačni rezultat.


In [49]:
# Bagging meta-estimator
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1), random_state=1)

model.fit(x_train, y_train)
model.score(x_test,y_test)


0.73983739837398377

In [50]:
# Bagging meta-estimator - kod za regresijski problem
from sklearn.ensemble import BaggingRegressor

model = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1), random_state=1)
model.fit(x_train_r, y_train_r)
model.score(x_test_r, y_test_r)


0.25271636220069549

## 2.2 Random forest

Random Forest je još jedan algoritam strojnog učenja koji slijedi *bagging* tehniku. To je proširenje *bagging meta-estimator* algoritma. Osnovni procjenitelji u *random forest*-u su stabla odlučivanja. Za razliku od *bagging meta-estimator*, *random forest* nasumično odabire skup značajki koje se koriste za određivanje najbolje podjele na svakom čvoru stabla odlučivanja. 


Ako ga gledamo korak po korak, ovo *random forest* model radi:

1. Stvaraju se slučajni podskupovi iz izvornog skupa podataka (bootstrapping).
2. Na svakom čvoru u stablu odlučivanja smatra se da slučajni skup značajki odlučuje o najboljoj podjeli.
3. Na svakom podskupu je ugrađen model stabla odlučivanja.
4. Konačno predviđanje izračunava se prosječnim predviđanjima svih stabala odlučivanja.


In [51]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)


0.76422764227642281

In [52]:
# Random forest - kod za regresijski problem
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=1)
model.fit(x_train_r, y_train_r)
model.score(x_test_r, y_test_r)


0.2828047197585537

# 3. Boosting methods


*Boosting ensemble* algoritmi kreiraju niz modela koji pokušavaju ispraviti pogreške prethodnog modela u nizu. Kada se modeli izgrade, naprave se predikcije koje mogu biti težinski skalirane prema točnosti modela, naposljetku se rezultati kombiniraju kako bi se stvorilo konačno predviđanje izlaza.

1. Kreira se podskup iz izvornog skupa podataka
2. Inicijalno su svim točkama dodijeljene jednake težine
3. Kreira se bazni model na tom podskupu
4. Ovaj se model koristi za predikciju cijelog skupa podataka
<div style="width:70%;margin:10px auto;">![boosting_1.png](img/boosting_1.png)</div>
5. Izračunavaju se pogreške koristeći stvarne i predviđene vrijednosti
6. Nepravilno predviđenim opservacijama se težine povećavaju (ovdje će tri pogrešno klasificirane plave plus točke dobiti veću težinu)
7. Kreira se drugi model i rade se predikcije na skupu podataka (ovaj model pokušava ispraviti pogreške iz prethodnog modela)
<div style="width:70%;margin:10px auto;">![boosting_2.png](img/boosting_2.png)</div>
8. Slično tome, kreira se više modela, svaki ispravljajući pogreške prethodnog modela.
9. Konačni model (snažan učenik) je aritmetička sredina rezultata svih modela (slabi učenici) množenim sa njegovom težinom 
<div style="width:70%;margin:10px auto;">![boosting_3.png](img/boosting_3.png)</div>


---


## 3.1. AdaBoost


Adaptive boosting ili AdaBoost je jedan od najjednostavnijih *boostring* algoritama. Obično se za modeliranje koriste stabla odluke. Stvaraju se višestruki sekvencijalni modeli, od kojih svaki ispravlja pogreške iz posljednjeg modela. AdaBoost dodjeljuje težine opservacijama koje su pogrešno predviđene, a naknadni model ispravno predviđa te vrijednosti. 


U nastavku su navedeni koraci za izvođenje AdaBoost algoritma:

1. U početku, sve opservacije u skupu podataka dobivaju jednake težine.
2. Model je izgrađen na podskupu podataka.
3. Koristeći ovaj model, radi se predikcija na cijelom skupu podataka.
4. Izračunavaju se pogreške usporedbom predviđanja i stvarnih vrijednosti.
5. Prilikom stvaranja sljedećeg modela se opservacijama koje su pogrešno predviđene dodaju veće težine.
6. Težine se mogu odrediti pomoću vrijednosti pogreške. Na primjer, veća pogreška -> veća težina dodijeljena opservaciji.
7. Taj se postupak ponavlja sve dok se funkcija pogreške ne promijeni ili se ne postigne maksimalno ograničenje broja procjenitelja.


In [53]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)


0.82113821138211385

In [54]:
# AdaBoost - kod za regresijski problem
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(random_state=1)
model.fit(x_train_r, y_train_r)
model.score(x_test_r, y_test_r)


0.3442780301954409

## 3.2. Gradient Boosting (GBM)


Gradient Boosting ili GBM je još jedan algoritam strojnog učenja koji radi za probleme regresije i klasifikacije. GBM koristi *boosting* tehniku, kombinirajući brojne slabe učenike kako bi oblikovali snažnog učenika. Stabla regresije koji se koriste kao osnovni učenici ima određenu pogrešku, te je svako sljedeće stablo u nizu izgrađeno na pogreškama koje je izračunalo prethodni stablo. 



In [55]:
# Gradient Boosting (GBM)

from sklearn.ensemble import GradientBoostingClassifier

model= GradientBoostingClassifier(learning_rate=0.01, random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)


0.82926829268292679

In [56]:
# Gradient Boosting (GBM) - kod za regresijski problem
from sklearn.ensemble import GradientBoostingRegressor

model= GradientBoostingRegressor(random_state=1)
model.fit(x_train_r, y_train_r)
model.score(x_test_r, y_test_r)


0.39232465088155677

## 3.3. XGBoost


XGBoost (extreme Gradient Boosting) je napredna implementacija *gradient boosting* algoritma. XGBoost se pokazao kao vrlo učinkovit ML algoritam, široko korišten u natjecanjima strojnog učenja i hackathonima. XGBoost ima visoku prediktivnu snagu i gotovo je 10 puta brži od drugih gradijentnih tehnika. XGBoost također uključuje niz regulacija koje smanjuju *overfitting* i poboljšavaju ukupnu učinkovitost algoritma.

XGBoost ima ugrađenu rutinu za obradu nedostajućih vrijednosti tako da prilikom pripreme podataka nije potrebno sređivati nedostajuće vrijednosti.


In [57]:
# XGBoost
import xgboost as xgb

model = xgb.XGBClassifier(learning_rate=0.01, random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)


0.82926829268292679

In [58]:
# XGBoost - kod za regresijski problem
import xgboost as xgb

model = xgb.XGBRegressor(random_state=1)
model.fit(x_train_r, y_train_r)
model.score(x_test_r, y_test_r)


0.42269957094733496

## 3.4. CatBoost

Rad sa kategoričkim varijablama je zamoran proces, osobito ako imate velik broj takvih varijabli. Kada vaše kategorijske varijable imaju previše oznaka (tj. Visoko su kardinalne), izvođenje *one-hot encoding*-a na njima eksponencijalno povećava dimenzionalnost i postaje stvarno teško raditi s skupom podataka.

CatBoost se može automatski baviti kategorijskim varijablama i ne zahtijeva opsežnu obradu podataka kao i drugi algoritmi strojnog učenja. Dakle, nije potrebno izvoditi transformaciju kategoričkih varijabli, već je dovoljno učitati podatke, ispraviti nedostajuće vrijednosti i podaci su spremni.



In [59]:
# CatBoost
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_seed=1)
model.fit(x_train, y_train.astype('category').cat.codes, eval_set=(x_test, y_test.astype('category').cat.codes), verbose=False)
model.score(x_test, y_test.astype('category').cat.codes)

0.83739837398373984

In [60]:
# CatBoost - kod za regresijski problem
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_seed=1)
model.fit(x_train_r, y_train_r, eval_set=(x_test_r, y_test_r), verbose=False)
model.score(x_test_r, y_test_r)

56.221165477328917